# FR Assessment
I have created this ipynb file which can be used as a Google colab notebook to deploy an ec2 instance on cloud.

This notebook performs below steps
1. Run your program
2. Deploy the virtual machine
3. SSH into the instance as user1 and user2
4. Read from and write to each of two volumes

We have used AWS programmatic access for connecting with AWS and used libraries such as boto3, parimiko for ssh onto the environment

In [15]:
# If deploying using Google Colan below code will be used, else comment this code. And deploy using python script
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
#Set your project path 
project_path = '/content/drive/My Drive/FR_Assessment/'

In [2]:
pip install awscli boto3 ##Used to connect with awscli using boto3


     |████████████████████████████████| 3.6MB 12.6MB/s 
     |████████████████████████████████| 133kB 61.5MB/s 
     |████████████████████████████████| 7.4MB 39.8MB/s 
     |████████████████████████████████| 552kB 29.6MB/s 
     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 153kB 48.5MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: docutils 0.17
    Uninstalling docutils-0.17:
      Successfully uninstalled docutils-0.17


In [3]:
pip install paramiko ## ssh client for python to connect with ec2

     |████████████████████████████████| 215kB 10.6MB/s 
     |████████████████████████████████| 3.2MB 18.0MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 962kB 54.2MB/s 


In [19]:
import boto3

## ACCESS_KEY and SECRET ACCESS_KEY for account where you need to deploy the code is to be added in below variables
aws_access_key_id = 'ACCESS_KEY'
aws_secret_access_key = 'SECRET_KEY'
region_name = 'us-east-1'

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key,
                                region_name=region_name)

ec2client = session.client('ec2')

In [ ]:
# create an EBS volume, 10G size
ebs_vol = ec2client.create_volume(
        Size=10,
        AvailabilityZone='us-east-1a',
        VolumeType='gp2'

    )

volume_id = ebs_vol['VolumeId']

# check that the EBS volume has been created successfully
if ebs_vol['ResponseMetadata']['HTTPStatusCode'] == 200:
  print ("Successfully created Volume! " + volume_id)

In [ ]:
import boto3

user_data = '''<script>
#!/bin/bash
echo 'test' > /tmp/hello
</script>'''

# create a new EC2 instance
ec2_instance = ec2client.run_instances(
     ImageId='ami-0742b4e673072066f', 
     MinCount=1,
     MaxCount=1,
     InstanceType='t2.micro',
     KeyName='FRkeypair.pem',
     Placement={'AvailabilityZone': 'us-east-1a'},
     UserData=user_data
 )

# check if EC2 instance was created successfully
if ec2_instance['ResponseMetadata']['HTTPStatusCode'] == 200:
  print ("Successfully created instance! " + ec2_instance['Instances'][0]['InstanceId'])

In [24]:
# attaching EBS volume to our EC2 instance
attach_resp = ec2client.attach_volume(
  VolumeId=volume_id,
  InstanceId=ec2_instance['Instances'][0]['InstanceId'],
  Device='/dev/xvdb'
    )

In [25]:
def ssh_connect_with_retry(ssh, ip_address, retries):
    if retries > 3:
        return False
    privkey = paramiko.RSAKey.from_private_key_file(
        'FRkeypair.pem')
    interval = 5
    try:
        retries += 1
        print('SSH into the instance: {}'.format(ip_address))
        ssh.connect(hostname=ip_address,
                    username='ubuntu', pkey=privkey)
        return True
    except Exception as e:
        print(e)
        time.sleep(interval)
        print('Retrying SSH connection to {}'.format(ip_address))
        ssh_connect_with_retry(ssh, ip_address, retries)


In [ ]:
import paramiko
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
## ssh onto ec2 instance
ip_address =  ec2_instance.get['PublicIpAddress']
ssh_connect_with_retry(ssh, ip_address, 0)

stdin, stdout, stderr = ssh.exec_command(commands)
print('stdout:', stdout.read())
print('stderr:', stderr.read())